First import pantheon and asyncio, and initiate a Pantheon instance

In [1]:
from pantheon import pantheon
import asyncio

server = "euw1"
api_key = "RGAPI-XXXX"

panth = pantheon.Pantheon(server, api_key, True)

Create the function to get game data

In [2]:

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":10})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.getMatch(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)

Get you games data

In [3]:
name = "Canisback"

loop = asyncio.get_event_loop()  

(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))
gamesDataRaw = loop.run_until_complete(getRecentMatches(accountId))
print(gamesDataRaw[0])

{'mapId': 11, 'participants': [{'timeline': {'role': 'DUO_CARRY', 'goldPerMinDeltas': {'10-20': 269.7, '0-10': 186.9}, 'xpPerMinDeltas': {'10-20': 289.9, '0-10': 241.8}, 'lane': 'BOTTOM', 'csDiffPerMinDeltas': {'10-20': -2.4, '0-10': -2.8000000000000007}, 'creepsPerMinDeltas': {'10-20': 5.3, '0-10': 4}, 'damageTakenPerMinDeltas': {'10-20': 484.8, '0-10': 271.6}, 'damageTakenDiffPerMinDeltas': {'10-20': 48.150000000000006, '0-10': -9.549999999999997}, 'participantId': 1, 'xpDiffPerMinDeltas': {'10-20': -273.49999999999994, '0-10': -28.64999999999995}}, 'highestAchievedSeasonTier': 'SILVER', 'stats': {'damageDealtToObjectives': 1258, 'totalScoreRank': 0, 'combatPlayerScore': 0, 'totalUnitsHealed': 2, 'perk1Var2': 0, 'largestKillingSpree': 0, 'perkSubStyle': 8100, 'trueDamageDealt': 1680, 'perk5Var1': 1, 'visionScore': 5, 'neutralMinionsKilled': 4, 'damageSelfMitigated': 7288, 'playerScore7': 0, 'totalPlayerScore': 0, 'firstTowerKill': False, 'largestMultiKill': 1, 'magicalDamageTaken': 5

The first step in this analysis is to figure out what is the participantId of the targeted player.

In [4]:
def getParticipantIdFromAccountId(game,accountId):
    for participant in game['participantIdentities']:
        if participant['player']['currentAccountId'] == accountId:
            return participant['participantId']
        
participantId = getParticipantIdFromAccountId(gamesDataRaw[0],accountId)
participantId

8

Now our targeted data are champion played by the player, and team composition. First, we get to now what champion has our player and in which team he is, and if he won.

In [5]:
def getTeamAndChampion(game, participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            return participant['teamId'],participant['championId'],participant['stats']['win']

teamId, championId, win = getTeamAndChampion(gamesDataRaw[0], participantId)
print(teamId, championId, win)

200 89 True


A little trick to know for teamId is that any summoner with participantId from 1 to 5 is in team 100 and from 6 to 10 in team 200, but it's only for 5v5 games.

Next is getting team composition.

In [6]:
def getTeamComposition(game):
    #Create an array for each team
    compositions = {100:[],200:[]}
    for participant in game['participants']:
        compositions[participant['teamId']].append(participant['championId'])
    return compositions

compositions = getTeamComposition(gamesDataRaw[0])
compositions

{100: [236, 77, 7, 497, 79], 200: [51, 86, 89, 69, 107]}

Now that we have all team compositions, we would want to have their name instead of championId. Let's use ddragon to achieve that by creating a championId=>name dictionnary

In [7]:
import requests, json
response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.9.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championIdToName = {}
for key,champion in champonRawData['data'].items():
    championIdToName[int(champion['key'])] = champion['name']
championIdToName

{1: 'Annie',
 2: 'Olaf',
 3: 'Galio',
 4: 'Twisted Fate',
 5: 'Xin Zhao',
 6: 'Urgot',
 7: 'LeBlanc',
 8: 'Vladimir',
 9: 'Fiddlesticks',
 10: 'Kayle',
 11: 'Master Yi',
 12: 'Alistar',
 13: 'Ryze',
 14: 'Sion',
 15: 'Sivir',
 16: 'Soraka',
 17: 'Teemo',
 18: 'Tristana',
 19: 'Warwick',
 20: 'Nunu',
 21: 'Miss Fortune',
 22: 'Ashe',
 23: 'Tryndamere',
 24: 'Jax',
 25: 'Morgana',
 26: 'Zilean',
 27: 'Singed',
 28: 'Evelynn',
 29: 'Twitch',
 30: 'Karthus',
 31: "Cho'Gath",
 32: 'Amumu',
 33: 'Rammus',
 34: 'Anivia',
 35: 'Shaco',
 36: 'Dr. Mundo',
 37: 'Sona',
 38: 'Kassadin',
 39: 'Irelia',
 40: 'Janna',
 41: 'Gangplank',
 42: 'Corki',
 43: 'Karma',
 44: 'Taric',
 45: 'Veigar',
 48: 'Trundle',
 50: 'Swain',
 51: 'Caitlyn',
 53: 'Blitzcrank',
 54: 'Malphite',
 55: 'Katarina',
 56: 'Nocturne',
 57: 'Maokai',
 58: 'Renekton',
 59: 'Jarvan IV',
 60: 'Elise',
 61: 'Orianna',
 62: 'Wukong',
 63: 'Brand',
 64: 'Lee Sin',
 67: 'Vayne',
 68: 'Rumble',
 69: 'Cassiopeia',
 72: 'Skarner',
 74: 'Hei

In [8]:
playerChampion = championIdToName[championId]
playerChampion

'Leona'

In [9]:
teamComposition = [championIdToName[chId] for chId in compositions[teamId]]
teamComposition

['Caitlyn', 'Garen', 'Leona', 'Cassiopeia', 'Rengar']

In [10]:
enemyTeamComposition = [championIdToName[chId] for chId in compositions[100 if teamId==200 else 200]]
enemyTeamComposition

['Lucian', 'Udyr', 'LeBlanc', 'Rakan', 'Gragas']

You can also transform the array in a single string

In [11]:
"/".join(teamComposition)

'Caitlyn/Garen/Leona/Cassiopeia/Rengar'

In [12]:
"/".join(enemyTeamComposition)

'Lucian/Udyr/LeBlanc/Rakan/Gragas'

Add some other data like patch and date

In [13]:
def getPatch(game):
    #Game version number contains minor versions, only take the two major ones.
    return ".".join(game['gameVersion'].split(".")[:2])
getPatch(gamesDataRaw[0])

'8.8'

In [14]:
import datetime

def getDate(game):
    return datetime.datetime.fromtimestamp(int(game["gameCreation"]/1000)).strftime('%Y/%m/%d')

getDate(gamesDataRaw[0])

'2018/05/01'

Now that we have all the data we need (for now), let's generalize it for every game we get

In [15]:
gameInformationList = []

for game in gamesDataRaw:
    #Get the information
    participantId = getParticipantIdFromAccountId(game,accountId)
    teamId, championId, win = getTeamAndChampion(game, participantId)
    compositions = getTeamComposition(game)
    
    #Select information you want to show
    gameInformation = {}
    gameInformation['win'] = win
    gameInformation['champion'] = championIdToName[championId]
    gameInformation['patch'] = getPatch(game)
    gameInformation['date'] = getDate(game)
    gameInformation['teamComposition'] = "/".join([championIdToName[chId] for chId in compositions[teamId]])
    gameInformation['enemyTeamComposition'] = "/".join([championIdToName[chId] for chId in compositions[100 if teamId==200 else 200]])
    #Add the row to the list
    gameInformationList.append(gameInformation)

gameInformationList

[{'champion': 'Leona',
  'date': '2018/05/01',
  'enemyTeamComposition': 'Lucian/Udyr/LeBlanc/Rakan/Gragas',
  'patch': '8.8',
  'teamComposition': 'Caitlyn/Garen/Leona/Cassiopeia/Rengar',
  'win': True},
 {'champion': 'Leona',
  'date': '2018/05/01',
  'enemyTeamComposition': 'Lucian/Zed/Taric/Evelynn/Pantheon',
  'patch': '8.8',
  'teamComposition': 'Nasus/Garen/LeBlanc/Lee Sin/Leona',
  'win': False},
 {'champion': 'Garen',
  'date': '2018/05/01',
  'enemyTeamComposition': 'Kennen/Lee Sin/Viktor/Thresh/Draven',
  'patch': '8.8',
  'teamComposition': 'Rakan/Xin Zhao/Jhin/Brand/Garen',
  'win': True},
 {'champion': 'Ziggs',
  'date': '2018/05/01',
  'enemyTeamComposition': 'Yasuo/Varus/Dr. Mundo/Jax/Zyra',
  'patch': '8.8',
  'teamComposition': 'Xin Zhao/Jhin/Brand/Ziggs/Jax',
  'win': True},
 {'champion': 'Ziggs',
  'date': '2018/05/01',
  'enemyTeamComposition': "Nasus/Miss Fortune/Lux/Kai'Sa/Katarina",
  'patch': '8.8',
  'teamComposition': 'Heimerdinger/Xin Zhao/Jhin/Brand/Ziggs',

Great, you have your data! I recommend you to use pandas to display nd manipulate the data easily

In [16]:
import pandas as pd
df = pd.DataFrame(gameInformationList)
df

,champion,date,enemyTeamComposition,patch,teamComposition,win
0,Leona,2018/05/01,Lucian/Udyr/LeBlanc/Rakan/Gragas,8.8,Caitlyn/Garen/Leona/Cassiopeia/Rengar,True
1,Leona,2018/05/01,Lucian/Zed/Taric/Evelynn/Pantheon,8.8,Nasus/Garen/LeBlanc/Lee Sin/Leona,False
2,Garen,2018/05/01,Kennen/Lee Sin/Viktor/Thresh/Draven,8.8,Rakan/Xin Zhao/Jhin/Brand/Garen,True
3,Ziggs,2018/05/01,Yasuo/Varus/Dr. Mundo/Jax/Zyra,8.8,Xin Zhao/Jhin/Brand/Ziggs/Jax,True
4,Ziggs,2018/05/01,Nasus/Miss Fortune/Lux/Kai'Sa/Katarina,8.8,Heimerdinger/Xin Zhao/Jhin/Brand/Ziggs,True
5,Garen,2018/05/01,Ezreal/Caitlyn/Nami/Graves/Singed,8.8,Leona/Vel'Koz/Thresh/Miss Fortune/Garen,True
6,Teemo,2018/05/01,Lux/Morgana/Teemo/Brand/Kha'Zix,8.8,Vel'Koz/Kha'Zix/Miss Fortune/Teemo/Kayle,True
7,Teemo,2018/05/01,Aurelion Sol/Varus/Alistar/Jarvan IV/Camille,8.8,Garen/Jinx/Lulu/Teemo/Orianna,False
8,Teemo,2018/04/30,Jayce/Syndra/Kled/Cho'Gath/Ezreal,8.8,Lux/Jinx/Lulu/Teemo/Vi,True
9,Teemo,2018/04/30,Corki/Anivia/Ryze/Vladimir/Irelia,8.8,Xin Zhao/Caitlyn/Lulu/Teemo/Yasuo,True
